<a href="https://colab.research.google.com/github/adityamavle/Amazon-ML-Challenge/blob/main/AmazonChallenge23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Initial Imports

In [ ]:
import zipfile
!wget https://s3-ap-southeast-1.amazonaws.com/he-public-data/datasetb2d9982.zip -O AmazonChallenge23Dataset
zip_ref = zipfile.ZipFile("AmazonChallenge23Dataset")
zip_ref.extractall()
zip_ref.close()

--2023-04-22 12:18:32--  https://s3-ap-southeast-1.amazonaws.com/he-public-data/datasetb2d9982.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.128.134
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.128.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 895569552 (854M) [binary/octet-stream]
Saving to: ‘AmazonChallenge23Dataset’

AmazonChallenge23Da 100%[===================>] 854.08M  14.5MB/s    in 63s     

2023-04-22 12:19:35 (13.6 MB/s) - ‘AmazonChallenge23Dataset’ saved [895569552/895569552]



In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv("/content/dataset/train.csv")

In [ ]:
train.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000


In [ ]:
train.corr()

<ipython-input-4-0bc27f030cd9>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.corr()


,PRODUCT_ID,PRODUCT_TYPE_ID,PRODUCT_LENGTH
PRODUCT_ID,1.000000,0.250576,0.000424
PRODUCT_TYPE_ID,0.250576,1.000000,0.000961
PRODUCT_LENGTH,0.000424,0.000961,1.000000


### Imputing null values in bullet points and description

In [ ]:
train2 = train[(train["PRODUCT_ID"] >= 0) & (train["PRODUCT_ID"] <= 20000)]
train2.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
298,18716,In Defense of Globalization,NaN,In Defense of Globalization [Hardcover] [Jan 0...,115,787.400
857,9365,Virology: A Laboratory Manual (Instructor's Ma...,NaN,NaN,6264,850.000
872,14599,Henry V,NaN,NaN,113,507.873
971,1826,2023 Guide to the Night Sky: A month-by-month ...,NaN,NaN,6142,582.676
1168,6460,Office Management,NaN,NaN,75,129.921


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install summa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54408 sha256=46110b6731d2bf15e9646c71be44043f70270852d18667cf27aee9c7a27b01a7
  Stored in directory: /root/.cache/pip/wheels/ed/2c/5f/a0ccc5955d44d2cea78729f4425e73f818d2629517f7af0f8b
Successfully built summa


In [ ]:
### Function to replace null values by null string in a column

In [ ]:
train2['TITLE'].isna().value_counts()

False    15037
True         2
Name: TITLE, dtype: int64

In [ ]:
import nltk
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.tokenize import sent_tokenize
from summa import summarizer

nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

def remove_stopwords(text):
    # Tokenize text into words
    tokens = word_tokenize(text)
    # Filter out stop words
    tokens = [token for token in tokens if token not in stop_words]
    # Join tokens back into text
    text = ' '.join(tokens)
    return text

def extract_keywords(text):
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    # Calculate term frequency
    fdist = FreqDist(tokens)
    keywords = [word for word, freq in fdist.items() if freq > 1]
    return keywords

def summarize(text):
    summary = summarizer.summarize(text)
    return summary
for index, row in train2.iterrows():
  if not pd.isnull(row['TITLE']):
      title = row['TITLE']
      title = remove_stopwords(title)
      keywords = extract_keywords(title)
      if pd.isnull(row['BULLET_POINTS']):
          train2.at[index, 'BULLET_POINTS'] = ', '.join(keywords)
      # Remove stop words from DESCRIPTION column
      summary = summarize(title)
      # Replace null values in DESCRIPTION column with generated summary
      if pd.isnull(row['DESCRIPTION']):
          train2.at[index, 'DESCRIPTION'] = summary


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train2['DESCRIPTION'].isna().value_counts()

False    15037
True         2
Name: DESCRIPTION, dtype: int64

In [ ]:
train.shape[0]

2249698

In [ ]:
train[['DESCRIPTION', 'BULLET_POINTS', 'TITLE']].isnull().value_counts()

DESCRIPTION  BULLET_POINTS  TITLE
False        False          False    1038460
True         True           False     783497
             False          False     373872
False        True           False      53857
True         True           True          10
             False          True           2
dtype: int64

In [ ]:
print(train['DESCRIPTION'].isnull().value_counts())
print(train['BULLET_POINTS'].isnull().value_counts())


True     1157381
False    1092317
Name: DESCRIPTION, dtype: int64
False    1412334
True      837364
Name: BULLET_POINTS, dtype: int64


In [ ]:
train.isnull().sum()

PRODUCT_ID               0
TITLE                   12
BULLET_POINTS       837364
DESCRIPTION        1157381
PRODUCT_TYPE_ID          0
PRODUCT_LENGTH           0
dtype: int64

In [ ]:
# Count the number of rows where at least one column (DESCRIPTION, BULLET_POINTS, or TITLE) is null
null_rows = len(train[train[['DESCRIPTION', 'BULLET_POINTS', 'TITLE']].isnull().any(axis=1)])

# Print the count of null rows
print("Number of rows with at least one column (DESCRIPTION, BULLET_POINTS, or TITLE) as null: ", null_rows)


Number of rows with at least one column (DESCRIPTION, BULLET_POINTS, or TITLE) as null:  1211238


In [ ]:
for index, row in train2.iterrows():
    # Remove stop words from TITLE column
    desc = row['DESCRIPTION']
    #print(title)
    print(type(desc))

Streaming output truncated to the last 5000 lines.
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'str'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 

In [ ]:
# Define summarization function
def summarize(text):
    # Tokenize text into sentences
    sentences = sent_tokenize(text)
    # Calculate sentence scores based on term frequency
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    # Convert tfidf_matrix to dense array
    tfidf_matrix_dense = tfidf_matrix.toarray()
    # Calculate sentence scores using cosine distance
    sentence_scores = []
    for i in range(tfidf_matrix_dense.shape[0]):
        for j in range(tfidf_matrix_dense.shape[0]):
            if i != j:
                sentence_scores.append(cosine_distance(tfidf_matrix_dense[i], tfidf_matrix_dense[j]))
            else:
                sentence_scores.append(0) # Setting similarity score to 0 for the same sentence
    # Sort sentences by scores
    sorted_indices = sorted(range(len(sentence_scores)), key=lambda x: sentence_scores[x])
    # Select top sentences as summary
    summary = [sentences[i] for i in sorted_indices[:5]]  # Select top 5 sentences as summary
    return " ".join(summary)


In [ ]:
train_dropped = train.dropna()
train_dropped.shape[0]

1038460

### Training a baseline distilbert

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
train_dropped = train.dropna()
train_dropped = train_dropped.sample(n=10000,random_state=52)

In [ ]:
import numpy as np

# Assuming train_input_ids, train_attention_masks, and train_labels are NumPy arrays

# Check shape of train_input_ids
print("Shape of train_input_ids:", train_input_ids.shape)

# Check shape of train_attention_masks
print("Shape of train_attention_masks:", train_attention_masks.shape)

# Check shape of train_labels
print("Shape of train_labels:", train_labels.shape)


Shape of train_input_ids: (8000,)
Shape of train_attention_masks: (8000,)
Shape of train_labels: (8000,)


In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
max_length = 128
max_seq_length = 128
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Step 2: Load and preprocess the dataset
# Assume that the dataset is loaded into a pandas DataFrame called "df"
# and that the "bullet_points" column contains the bullet points and "product_length" contains the label

X = train_dropped['BULLET_POINTS'].tolist()
y = train_dropped['PRODUCT_LENGTH'].values

# Preprocess the bullet points using the tokenizer
X_preprocessed = []
for text in X:
    input_ids = tokenizer.encode(text, add_special_tokens=True,padding = 'max_length',max_length = max_length,truncation = True) #Truncation done
    X_preprocessed.append(input_ids)
    # Pad or truncate the input sequence to a fixed length
    if len(input_ids) > max_seq_length:
        input_ids = input_ids[:max_seq_length-1] + [tokenizer.sep_token_id]  # Truncate and add SEP token
    else:
        padding_length = max_seq_length - len(input_ids)
        input_ids = input_ids + [tokenizer.pad_token_id] * padding_length

In [ ]:
print(val_dataset)
print(train_input_ids.shape)

(8000, 200)


In [ ]:
val_df['length'] = val_df['BULLET_POINTS'].apply(lambda x: len(x))

In [ ]:
val_df['length'].value_counts()

200    2000
Name: length, dtype: int64

In [ ]:
print(val_df['BULLET_POINTS'])

736700     [101, 1031, 7427, 2443, 1024, 1015, 1060, 1294...
91546      [101, 1031, 2715, 1011, 4906, 5413, 2080, 2460...
44777      [101, 1031, 18546, 3756, 1049, 2575, 2581, 161...
60815      [101, 1031, 2308, 1005, 1055, 3614, 1024, 1016...
2197343    [101, 1031, 5308, 1997, 1017, 9132, 1010, 2946...
                                 ...                        
1101737    [101, 1031, 1026, 1038, 1028, 3430, 1026, 1013...
737079     [101, 1031, 9381, 1011, 2184, 4960, 1010, 4578...
1853848    [101, 1031, 4633, 1011, 6947, 1998, 2300, 1011...
374178     [101, 1031, 1063, 7661, 2569, 9604, 1065, 1011...
671268     [101, 1031, 3602, 1024, 1996, 10826, 3855, 268...
Name: BULLET_POINTS, Length: 2000, dtype: object


In [ ]:
class DistilBERTRegression(torch.nn.Module):
    def __init__(self):
        super(DistilBERTRegression, self).__init__()
        self.distilbert = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
        self.linear = torch.nn.Linear(768, 1)  # 768 is the hidden size of DistilBERT

    def forward(self, input_ids,attention_masks):
        outputs = self.distilbert(input_ids)
        pooled_output = outputs[0][:, 0]  # Extract the pooled output of the first token
        prediction = self.linear(pooled_output)
        return prediction.view(-1)

# Instantiate the model
model = DistilBERTRegression()

# Define training arguments
training_args = TrainingArguments(
output_dir='./results', # Directory for storing training outputs
num_train_epochs=3, # Number of epochs for training
per_device_train_batch_size=8, # Batch size for training
per_device_eval_batch_size=8, # Batch size for evaluation
save_steps=-1, # Number of steps to save model checkpoint
save_total_limit=2, # Limit on the total number of saved checkpoints
prediction_loss_only=True, # Compute only prediction loss, not other losses
logging_dir='./logs', # Directory for storing logs
logging_steps=500, # Number of steps for logging
evaluation_strategy='epoch',
save_strategy = 'epoch',
load_best_model_at_end=True, # Load the best model at the end of training
metric_for_best_model='eval_loss', # Metric for selecting the best model
greater_is_better=False # Whether a greater value of the metric is better or not
)
# training_args = TrainingArguments(
#     per_device_train_batch_size=8,  # Batch size for training
#     num_train_epochs=3,  # Number of training epochs
#     save_steps=-1,  # Number of steps between each save
#     save_total_limit=2,  # Maximum number of saved checkpoints
#     output_dir='./output',  # Directory to save the model
#     overwrite_output_dir=True,  # Overwrite the output directory if it already exists
#     logging_dir='./logs',  # Directory to save the logs
#     logging_steps=500,  # Number of steps between each log
#     evaluation_strategy='epoch',
#     save_strategy = 'epoch',  # When to evaluate the model during training
#     load_best_model_at_end=True,  # Load the best model at the end of training
#     metric_for_best_model='eval_loss',  # Metric to determine the best model
#     greater_is_better=False  # Smaller evaluation metric is better (since it's a regression problem)
# )

def compute_metrics(p: EvalPrediction):
  preds = p.predictions.flatten()
  labels = p.label_ids.flatten()
  return {
  'mae': mean_absolute_error(labels, preds)
  }

#Create data collator
data_collator = DataCollatorForLanguageModeling(tokenizer)

#Create Trainer
trainer = Trainer(
  model=model,
  args=training_args,
  data_collator=data_collator,
  train_dataset=train_dataset,
  eval_dataset=val_dataset,
  compute_metrics=compute_metrics,
)

#Train the model
trainer.train()

#Evaluate the model
eval_results = trainer.evaluate(eval_dataset=val_dataset)
print(f'Evaluation results: {eval_results}')


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

ValueError: ignored

In [ ]:
from torch.nn.utils.rnn import pad_sequence
import torch

# Convert numpy arrays to list of tensors
train_input_ids_list = [torch.tensor(seq) for seq in train_input_ids]
train_attention_masks_list = [torch.tensor(seq) for seq in train_attention_masks]

# Pad sequences to the maximum length
train_input_ids_padded = pad_sequence(train_input_ids_list, batch_first=True, padding_value=0)
train_attention_masks_padded = pad_sequence(train_attention_masks_list, batch_first=True, padding_value=0)

val_input_ids_list = [torch.tensor(seq) for seq in val_input_ids]
val_attention_masks_list = [torch.tensor(seq) for seq in val_attention_masks]

# Pad sequences to the maximum length
val_input_ids_padded = pad_sequence(val_input_ids_list, batch_first=True, padding_value=0)
val_attention_masks_padded = pad_sequence(val_attention_masks_list, batch_first=True, padding_value=0)
# Verify the shapes of padded sequences
print("Shape of train_input_ids_padded:", train_input_ids_padded.shape)
print("Shape of train_attention_masks_padded:", train_attention_masks_padded.shape)
val_input_ids_list = [torch.tensor(seq) for seq in val_input_ids]
val_attention_masks_list = [torch.tensor(seq) for seq in val_attention_masks]

# Pad sequences to the maximum length
print("Shape of val_input_ids_padded:", val_input_ids_padded.shape)
print("Shape of val_attention_masks_padded:", val_attention_masks_padded.shape)

Shape of train_input_ids_padded: torch.Size([8000, 152])
Shape of train_attention_masks_padded: torch.Size([8000, 152])
Shape of val_input_ids_padded: torch.Size([2000, 155])
Shape of val_attention_masks_padded: torch.Size([2000, 155])


In [ ]:
train_input_ids.shape,val_input_ids.shape

((8000, 200), (2000, 200))

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, DistilBertConfig
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from transformers import EvalPrediction
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import mean_absolute_error
import torch
from transformers import DataCollatorForLanguageModeling
config = DistilBertConfig.from_pretrained('distilbert-base-uncased', num_labels=1)#Regression problem,num_labels=1
# Create a DistilBERT model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', config=config)
# Define the training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=8,  # Batch size for training
    num_train_epochs=3,  # Number of training epochs
    save_steps=-1,  # Number of steps between each save
    save_total_limit=2,  # Maximum number of saved checkpoints
    output_dir='./output',  # Directory to save the model
    overwrite_output_dir=True,  # Overwrite the output directory if it already exists
    logging_dir='./logs',  # Directory to save the logs
    logging_steps=500,  # Number of steps between each log
    evaluation_strategy='epoch',
    save_strategy = 'epoch',  # When to evaluate the model during training
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model='eval_loss',  # Metric to determine the best model
    greater_is_better=False  # Smaller evaluation metric is better (since it's a regression problem)
)
# class MyDataCollator(DataCollatorForLanguageModeling):
#     def collate_batch(self, examples):
#         if isinstance(examples[0], (list, tuple)):
#             # If examples is a list of examples
#             batch = super().collate_batch(examples)
#         else:
#             # If examples is a single example
#             batch = {k: torch.unsqueeze(v, 0) for k, v in examples.items()}

#         # Convert tensors to numpy arrays to avoid vars() error
#         batch = {k: v.numpy() if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
#         return batch
# data_collator = MyDataCollator(tokenizer=tokenizer, mlm=False)

#Create a Trainer instance
def compute_metrics(eval_pred):
     predictions = eval_pred.predictions.flatten()
     labels = eval_pred.label_ids.flatten()
     metrics = {
         'mae': mean_absolute_error(labels, predictions)  # Mean Absolute Error
     }
     return metrics

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics = compute_metrics
)

# Define a function to compute metrics during evaluation
# Train the model
trainer.train()

# Visualize the metrics using Tensorboard
writer = SummaryWriter(log_dir='./logs')  # Create a SummaryWriter instance
writer.add_scalar('Train Loss', trainer.train_dataloader.dataset.metrics['train_loss'].get_latest(), global_step=trainer.global_step)  # Log train loss
writer.add_scalar('Eval Loss', trainer.eval_dataloader.dataset.metrics['eval_loss'].get_latest(), global_step=trainer.global_step)  # Log eval loss
writer.add_scalar('MAE', trainer.eval_dataloader.dataset.metrics['mae'].get_latest(), global_step=trainer.global_step)  # Log MAE
writer.flush()  # Flush the writer
writer.close()  # Close the writer


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

ValueError: ignored

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('/content/dataset/train.csv')
train_dropped = train.dropna()
train_dropped = train_dropped.sample(n=10000)

In [ ]:
del train

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, DistilBertConfig
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from transformers import EvalPrediction
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import mean_absolute_error
import torch
from transformers import DataCollatorForLanguageModeling
# Split the dataset into train and validation sets
train_df, val_df = train_test_split(train_dropped, test_size=0.2, random_state=42)

# Load the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
max_length = 50 #truncate and pad all to the same length
def preprocess_text(text):
    # Tokenize text
    tokenized_text = tokenizer.encode(text, truncation=True, padding="max_length", max_length=max_length)
    if len(tokenized_text) < max_length:
       tokenized_text += [tokenizer.pad_token_id] * (max_length - len(tokenized_text))
    elif len(tokenized_text) > max_length:
       tokenized_text = tokenized_text[:max_length]
    return tokenized_text

train_df['TITLE'] = train_df['TITLE'].apply(preprocess_text)
train_df['DESCRIPTION'] = train_df['DESCRIPTION'].apply(preprocess_text)
train_df['BULLET_POINTS'] = train_df['BULLET_POINTS'].apply(preprocess_text)
val_df['TITLE'] = val_df['TITLE'].apply(preprocess_text)
val_df['DESCRIPTION'] = val_df['DESCRIPTION'].apply(preprocess_text)
val_df['BULLET_POINTS'] = val_df['BULLET_POINTS'].apply(preprocess_text)

def create_input_tensors(df):
    input_ids = np.array(df['TITLE'].tolist())
    #attention_masks = np.array(df['TITLE'].apply(lambda x: [1] * len(x)).tolist())
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    attention_masks = np.array([[1 if token_id != tokenizer.pad_token_id else 0 for token_id in input_id] for input_id in input_ids])
    labels = np.array(df['PRODUCT_LENGTH'].tolist())  #'PRODUCT_LENGTH':label

    return input_ids, attention_masks, labels

train_input_ids, train_attention_masks, train_labels = create_input_tensors(train_df)
val_input_ids, val_attention_masks, val_labels = create_input_tensors(val_df)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __getitem__(self, index):
        return {
            'input_ids': torch.tensor(self.input_ids[index], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_masks[index], dtype=torch.long),
            'labels': torch.tensor(self.labels[index], dtype=torch.float)
        }

    def __len__(self):
        return len(self.input_ids)

# train_dataset = CustomDataset(train_input_ids_padded, train_attention_masks_padded, train_labels)
# val_dataset = CustomDataset(val_input_ids_padded, val_attention_masks_padded, val_labels)

train_dataset = CustomDataset(train_input_ids, train_attention_masks, train_labels)
val_dataset = CustomDataset(val_input_ids, val_attention_masks, val_labels)
print(val_df['BULLET_POINTS'])

1626747    [101, 1031, 2440, 3417, 8782, 12997, 2015, 265...
829528     [101, 1031, 2204, 3737, 1998, 7218, 2000, 2224...
58698      [101, 1031, 4368, 2100, 7216, 2038, 2196, 2246...
950472     [101, 1031, 1996, 4799, 3829, 15740, 2015, 202...
2070671    [101, 1011, 9078, 12259, 17072, 1011, 19942, 4...
                                 ...                        
1513105    [101, 1031, 100, 6581, 3430, 100, 1024, 6065, ...
805428     [101, 1031, 100, 2005, 4253, 1013, 25039, 2015...
1511021    [101, 1031, 2023, 2630, 19392, 6556, 3084, 200...
835328     [101, 1031, 18906, 4402, 8313, 1024, 7221, 540...
458093     [101, 1037, 2460, 1010, 2898, 12073, 2003, 505...
Name: BULLET_POINTS, Length: 2000, dtype: object


In [ ]:
train_input_ids[10],train_attention_masks[10]

(array([  101,  9805,  2213,  2078, 14030, 10140,  3336,  5061, 13383,
         2005,  3057,  1011,  1019,  1048,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]))

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, Dataset
from transformers import TrainingArguments
from sklearn.metrics import mean_absolute_error
from transformers import DataCollatorForLanguageModeling


# Define DistilBERT regression model
class DistilBERTRegression(nn.Module):
    def __init__(self):
        super(DistilBERTRegression, self).__init__()
        self.distilbert = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
        self.linear = nn.Linear(16,200)  # 768 is the hidden size of DistilBERT

    def forward(self, input_ids, attention_masks):
        outputs = self.distilbert(input_ids)
        pooled_output = outputs[0][:, 0]  # Extract the pooled output of the first token
        prediction = self.linear(pooled_output)
        return prediction.view(-1)


# Define custom dataset class
# Instantiate the model
model = DistilBERTRegression()

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',  # Directory for storing training outputs
    num_train_epochs=3,  # Number of epochs for training
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    save_steps=-1,  # Number of steps to save model checkpoint
    save_total_limit=2,  # Limit on the total number of saved checkpoints
    prediction_loss_only=True,  # Compute only prediction loss, not other losses
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=500,  # Number of steps for logging
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model='eval_loss',  # Metric for selecting the best model
    greater_is_better=False  # Whether a greater value of the metric is better or not
)

# Create data collator
data_collator = DataCollatorForLanguageModeling(tokenizer)
# Load and preprocess your training and validation data
# input_ids, attention_masks, and labels are assumed to be preprocessed and ready to use
batch_size = 16
# Create data loaders
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
val_sampler = SequentialSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=batch_size)
#print(input_ids.shape)
# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define optimizer and scheduler
# Define optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)  # AdamW optimizer
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)  # Learning rate scheduler

# Define loss function
loss_function = nn.L1Loss()

# Define function for computing mean absolute error
def compute_mae(predictions, labels):
    return mean_absolute_error(predictions.detach().cpu().numpy(), labels.detach().cpu().numpy())

# Training loop
best_val_mae = float('inf')  # Initialize best validation mean absolute error
for epoch in range(training_args.num_train_epochs):
    model.train()  # Set model to training mode
    train_loss = 0
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()  # Zero the gradients
        predictions = model(input_ids, attention_mask)  # Forward pass
        loss = loss_function(predictions, labels)  # Compute loss # Problem is in the shape of predictions and self.Linear
        loss.backward()  # Backward pass
        optimizer.step()  # Update model weights

        train_loss += loss.item()  # Accumulate training loss

    train_loss /= len(train_dataloader)  # Compute average training loss

    model.eval()  # Set model to evaluation mode
    val_loss = 0
    val_predictions = []
    val_labels = []
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.no_grad():  # Disable gradient computation
            predictions = model(input_ids, attention_mask)  # Forward pass
            loss = loss_function(predictions, labels)  # Compute loss

        val_loss += loss.item()  # Accumulate validation loss
        val_predictions.extend(predictions.detach().cpu().numpy())  # Collect predictions
        val_labels.extend(labels.detach().cpu().numpy())  # Collect labels

    val_loss /= len(val_dataloader)  # Compute average validation loss
    val_mae = compute_mae(torch.tensor(val_predictions), torch.tensor(val_labels))  # Compute validation mean absolute error

    # Update best validation mean absolute error and save model checkpoint
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        torch.save(model.state_dict(), 'best_model.pth')  # Save best model checkpoint

    print(f'Epoch {epoch + 1}:')
    print(f'Training Loss: {train_loss:.4f} | Validation Loss: {val_loss:.4f} | Validation MAE: {val_mae:.4f}')
    print('----------------------------------------')

    scheduler.step()  # Update learning rate scheduler

print('Training completed successfully!')

# Load the best model checkpoint
best_model = DistilBERTRegression()
best_model.load_state_dict(torch.load('best_model.pth'))
best_model.to(device)

# Use the best model for inference
# ...




Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

RuntimeError: ignored

In [ ]:
print(train_dataloader)

In [ ]:
train_input_ids[0],train_labels[0]

([101,
  2821,
  2213,
  2047,
  2259,
  28799,
  6178,
  11756,
  4345,
  2566,
  29278,
  4383,
  22953,
  9077,
  3237,
  5898,
  6007,
  2005,
  2273,
  102],
 1338.5826758)

In [ ]:
train_input_ids.shape

(8000,)

In [ ]:
train_dropped.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
621456,976641,Fly Racing Two-Piece Rain Suit (XXXXX-Large) (...,Two-piece pant / jacket rain suit featuring hi...,Two-piece pant / jacket rain suit featuring hi...,8040,100.000000
1673737,1837214,ECCPP ABS Spoiler Wing Unpainted Rear Window R...,"[[FITMENT] Honda Civic 4-door 1.8L DX Sedan,[W...",<b>ECCPP provides many types of auto parts and...,7551,5098.425192
867132,1796993,"2020 Upgraded AirPods Pro Case, KMMIN Seamless...",[[ORIGINAL UNGRADED DESIGN] - Unique specially...,Have you misplaced one of your earhud in a jea...,1814,385.826771
492668,2675710,Vayuna Silk Solid Blackout Stainless Steel Rin...,[Product Name : Blackout Curtains For Window A...,About Vayuna Invite Vayuna products into your ...,1649,8385.826763
11762,1202129,INGEAR Crochet Casual Dress Embroidery Summer ...,[This Swimsuit cover up for women is made of s...,This beautiful umbrella print dress is soft to...,2986,1250.000000


In [ ]:
print(train_dataset)

In [ ]:
tensorboard --logdir=./logs

### BERT Model Regression

In [ ]:
import zipfile
!wget https://github.com/adityamavle/Amazon-ML-Challenge/blob/main/pre-proccessed.zip?raw=true -O AmazonChallenge23Cleaned
zip_ref = zipfile.ZipFile("AmazonChallenge23Cleaned")
zip_ref.extractall()
zip_ref.close()

--2023-04-23 09:44:29--  https://github.com/adityamavle/Amazon-ML-Challenge/blob/main/pre-proccessed.zip?raw=true
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/adityamavle/Amazon-ML-Challenge/raw/main/pre-proccessed.zip [following]
--2023-04-23 09:44:30--  https://github.com/adityamavle/Amazon-ML-Challenge/raw/main/pre-proccessed.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/adityamavle/Amazon-ML-Challenge/main/pre-proccessed.zip [following]
--2023-04-23 09:44:30--  https://raw.githubusercontent.com/adityamavle/Amazon-ML-Challenge/main/pre-proccessed.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.co

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('/content/pre-proccessed (1).csv')
train.head()

,Unnamed: 0,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,NUMERIC_DATA
0,1740317,2393940,Wobit Covers® - Yamaha YZF R1M New BS6 Water R...,"Wheeler,Resistant,Fabric,protection,Cover,Vehi...","customer,Dear,carefully,read,complete,descript...",7682,984.251967,NaN
1,1511738,2369752,Golden Cart Vertical Garden Artifical Mat with...,"5,QUANTITY,SIZE,Pieces,AREA,COVERAGE,ARTIFICIA...","Cart,Wall,Golden,background,Small,Grass,live,A...",1591,1614.173227,"2 ,2 ,50cm,50cm,5cm,5,5 ,50cm,50cm,5cm,13.45 ,..."
2,294232,1010041,"ARIAT Men's Logo Softshell Jacket, Black, Medium","Wind,collar,water,resistance,Stain,repellent,W...","wear,team,jacket,equestrian,athlete,aspires,zi...",2848,2600.000000,NaN
3,2032819,2496499,Ayesha Handloom Shaggy Carpet Plain Fur Rugs f...,"soft,rug,Luxurious,Ideal,2,Height,Ultra,Thickn...","room,Fluffy,rug,beauty,unbelievable,perfect,sp...",1636,2362.204722,"2 Inch,2 ,"
4,1042207,2302998,(Renewed) Dell Optiplex 380 Intel Core 2 Duo D...,"Intel,password,2,HDD,Enhanced,Technology,Bit,U...","Renewed,Amazon,duct,original,functional,purcha...",10449,1450.000000,"380 ,2 ,8 ,1 ,2 ,7200 ,"


In [ ]:
train.isna().sum()

Unnamed: 0             0
PRODUCT_ID             0
TITLE                  0
BULLET_POINTS        224
DESCRIPTION          159
PRODUCT_TYPE_ID        0
PRODUCT_LENGTH         0
NUMERIC_DATA       10155
dtype: int64

In [ ]:
import pandas as pd
import numpy as np
#train = pd.read_csv('/content/pre-processed.csv')
train_dropped_1 = train.dropna()
train_dropped = train_dropped_1.sample(n=70000, random_state=52)
train_dropped.isna().sum()

Unnamed: 0         0
PRODUCT_ID         0
TITLE              0
BULLET_POINTS      0
DESCRIPTION        0
PRODUCT_TYPE_ID    0
PRODUCT_LENGTH     0
NUMERIC_DATA       0
dtype: int64

In [ ]:
train_dropped = train_dropped.drop(['Unnamed: 0'], axis=1)

In [ ]:
train_dropped_1.to_csv('dropped_train.csv')


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
merged_text = train_dropped['TITLE'] + ' [CLS] ' + train_dropped['BULLET_POINTS'] + ' [CLS] ' + train_dropped['DESCRIPTION'] +  ' [SEP]'
mergd = merged_text.tolist()
mergd[0]

'jioeuinly Case for Cricket Icon 3 / ATT Motivate 2 / Cricket Splendor 4G LTE Phone Case Cover + [2 Pack] Film Soft TPU Screen Protector Blue [CLS] Cover,Film,Case,3,Soft,TPU,Pack,Phone,Screen,Cricket,ATT,Splendor,LTE,Icon,Motivate,Protector,jioeuinly,protection,equipment,Blue,Soft, [CLS] Pack,3,ATT,Splendor,LTE,Cover,Film,Blue,Case,Cricket,Icon,Motivate,Phone,Soft,TPU,Screen,Protector,jioeuinly, [SEP]'

In [ ]:
dropd = train_dropped['BULLET_POINTS'].tolist()
dropd[0]

'Cover,Film,Case,3,Soft,TPU,Pack,Phone,Screen,Cricket,ATT,Splendor,LTE,Icon,Motivate,Protector,jioeuinly,protection,equipment,Blue,Soft,'

In [ ]:
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
desc_title = train_dropped['DESCRIPTION'] + ' [SEP] ' + train_dropped['TITLE']

# Concatenate all four columns to create 'merged_text'
title_bullet_points = train_dropped['TITLE'] + ' [SEP] ' + train_dropped['BULLET_POINTS']
bullet_points_description = train_dropped['BULLET_POINTS'] + ' [SEP] ' + train_dropped['DESCRIPTION']
merged_text = title_bullet_points + ' [CLS] ' + bullet_points_description + ' [CLS] ' + desc_title
mergd = merged_text.tolist()
encoded_corpus = tokenizer(text=mergd,
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=300,
                            return_attention_mask=True)
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']
# merged_text = train_dropped['TRAIN'] + ' [CLS] ' + train_dropped['BULLET_POINTS'] + ' [CLS] ' + train_dropped['DESCRIPTION']

# X = X_text.tolist()

In [ ]:
print(input_ids[0])

[101, 10147, 8913, 20023, 2135, 2553, 2005, 4533, 12696, 1017, 1013, 2012, 2102, 9587, 29068, 3686, 1016, 1013, 4533, 11867, 7770, 7983, 1018, 2290, 8318, 2063, 3042, 2553, 3104, 1009, 1031, 1016, 5308, 1033, 2143, 3730, 1056, 14289, 3898, 16167, 2630, 102, 3104, 1010, 2143, 1010, 2553, 1010, 1017, 1010, 3730, 1010, 1056, 14289, 1010, 5308, 1010, 3042, 1010, 3898, 1010, 4533, 1010, 2012, 2102, 1010, 11867, 7770, 7983, 1010, 8318, 2063, 1010, 12696, 1010, 9587, 29068, 3686, 1010, 16167, 1010, 10147, 8913, 20023, 2135, 1010, 3860, 1010, 3941, 1010, 2630, 1010, 3730, 1010, 101, 3104, 1010, 2143, 1010, 2553, 1010, 1017, 1010, 3730, 1010, 1056, 14289, 1010, 5308, 1010, 3042, 1010, 3898, 1010, 4533, 1010, 2012, 2102, 1010, 11867, 7770, 7983, 1010, 8318, 2063, 1010, 12696, 1010, 9587, 29068, 3686, 1010, 16167, 1010, 10147, 8913, 20023, 2135, 1010, 3860, 1010, 3941, 1010, 2630, 1010, 3730, 1010, 102, 5308, 1010, 1017, 1010, 2012, 2102, 1010, 11867, 7770, 7983, 1010, 8318, 2063, 1010, 3104, 101

In [ ]:
# import numpy as np
# def filter_long_descriptions(tokenizer, descriptions, max_len):
#     indices = []
#     lengths = tokenizer(descriptions, padding=False,
#                      truncation=False, return_length=True)['length']
#     for i in range(len(descriptions)):
#         if lengths[i] <= max_len-2:
#             indices.append(i)
#     return indices
# short_descriptions = filter_long_descriptions(tokenizer,
#                               #  mergd, 300)
input_ids = np.array(input_ids)
attention_mask = np.array(attention_mask)
labels = train_dropped['PRODUCT_LENGTH'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
test_size = 0.1
seed = 42
train_inputs, test_inputs, train_labels, test_labels = \
            train_test_split(input_ids, labels, test_size=test_size,
                             random_state=seed)
train_masks, test_masks, _, _ = train_test_split(attention_mask,
                                        labels, test_size=test_size,
                                        random_state=seed)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# price_scaler = StandardScaler()
# price_scaler.fit(train_labels.reshape(-1, 1))
# train_labels = price_scaler.transform(train_labels.reshape(-1, 1))
# test_labels = price_scaler.transform(test_labels.reshape(-1, 1))

In [ ]:
train_inputs.shape

(63000, 300)

In [ ]:
# import torch
# from torch.utils.data import TensorDataset, DataLoader
# batch_size = 16
# def create_dataloaders(inputs, masks, labels, batch_size=16):
#     input_tensor = torch.tensor(inputs)
#     mask_tensor = torch.tensor(masks)
#     labels_tensor = torch.tensor(labels)
#     dataset = TensorDataset(input_tensor, mask_tensor,
#                             labels_tensor)
#     dataloader = DataLoader(dataset, batch_size=16,
#                             shuffle=True)
#     return dataloader
# train_dataloader = create_dataloaders(train_input_ids, train_attention_masks, train_labels)
# test_dataloader = create_dataloaders(val_input_ids, val_attention_masks, val_labels)

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
batch_size = 16
def create_dataloaders(inputs, masks, labels, batch_size):
    input_tensor = torch.tensor(inputs)
    mask_tensor = torch.tensor(masks)
    labels_tensor = torch.tensor(labels)
    dataset = TensorDataset(input_tensor, mask_tensor,
                            labels_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size,
                            shuffle=True)
    return dataloader
train_dataloader = create_dataloaders(train_inputs, train_masks,
                                      train_labels, batch_size)
test_dataloader = create_dataloaders(test_inputs, test_masks,
                                     test_labels, batch_size)

In [ ]:
import tensorflow as tf

batch_size = 16

def create_dataset(inputs, masks, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((inputs, masks, labels))
    dataset = dataset.shuffle(len(inputs)).batch(batch_size)
    return dataset

train_dataset = create_dataset(train_inputs, train_masks, train_labels, batch_size)
test_dataset = create_dataset(test_inputs, test_masks, test_labels, batch_size)

In [ ]:
# import torch.nn as nn
# from transformers import BertModel

# class BertRegressor(nn.Module):
#     def __init__(self, drop_rate=0.2, freeze_bert=False):
#         super(BertRegressor, self).__init__()
#         D_in, D_out = 768, 1
#         self.bert = BertModel.from_pretrained('bert-base-uncased')
#         self.regressor = nn.Sequential(
#             nn.Dropout(drop_rate),
#             nn.Linear(D_in, D_out))

#     def forward(self, input_ids, attention_masks):
#         outputs = self.bert(input_ids, attention_mask=attention_masks)
#         class_label_output = outputs[1]
#         outputs = self.regressor(class_label_output)
#         return outputs

# model = BertRegressor(drop_rate=0.2)

In [ ]:
# import torch.nn as nn
# from transformers import MobileBertModel

# class MobileBertRegressor(nn.Module):
#     def __init__(self, drop_rate=0.2, freeze_mobilebert=False):
#         super(MobileBertRegressor, self).__init__()
#         D_in, D_out = 512, 1  # Update input and output dimensions for MobileBertModel
#         self.mobilebert = MobileBertModel.from_pretrained('google/mobilebert-uncased')
#         self.regressor = nn.Sequential(
#             nn.Dropout(drop_rate),
#             nn.Linear(D_in, D_out))

#         # Freeze MobileBertModel if specified
#         if freeze_mobilebert:
#             for param in self.mobilebert.parameters():
#                 param.requires_grad = False

#     def forward(self, input_ids, attention_masks):
#         outputs = self.mobilebert(input_ids=input_ids, attention_mask=attention_masks)
#         class_label_output = outputs[1]  # Use the second element of the outputs tuple as the pooled output
#         outputs = self.regressor(class_label_output)
#         return outputs

# # Example usage
# model = MobileBertRegressor(drop_rate=0.2)

In [ ]:
# import torch.nn as nn
# from transformers import DistilBertModel

# class DistilBertRegressor(nn.Module):
#     def __init__(self, drop_rate=0.2, freeze_distilbert=False):
#         super(DistilBertRegressor, self).__init__()
#         D_in, D_out = 768, 1  # Update input and output dimensions for DistilBertModel
#         self.distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')
#         self.regressor = nn.Sequential(
#             nn.Dropout(drop_rate),
#             nn.Linear(D_in, D_out))

#         # Freeze DistilBertModel if specified
#         if freeze_distilbert:
#             for param in self.distilbert.parameters():
#                 param.requires_grad = False

#     def forward(self, input_ids, attention_masks):
#         outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_masks)
#         class_label_output = outputs[0][:, 0]  # Use the first element of the outputs tuple and take the first token [CLS] representation as the pooled output
#         outputs = self.regressor(class_label_output)
#         return outputs

# # Example usage
# model = DistilBertRegressor(drop_rate=0.2)

In [ ]:
next(iter(train_dataset))

(<tf.Tensor: shape=(16, 300), dtype=int64, numpy=
 array([[  101,  5631,  3509, ...,     0,     0,     0],
        [  101,  2553,  7529, ...,     0,     0,     0],
        [  101,  7861, 19279, ...,     0,     0,     0],
        ...,
        [  101,  2190, 27072, ...,     0,     0,     0],
        [  101, 16420,  2232, ...,     0,     0,     0],
        [  101, 24665, 16530, ...,     0,     0,     0]])>,
 <tf.Tensor: shape=(16, 300), dtype=int64, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])>,
 <tf.Tensor: shape=(16,), dtype=float64, numpy=
 array([ 196.8503935 ,  551.1811018 ,  200.        ,  100.        ,
         393.700787  , 1377.9527545 ,  700.        , 1400.        ,
         708.6614166 ,  669.29      , 2799.99999714,  393.700787  ,
         800.        ,  590.5511805 , 1889.7637776 ,  393.700787  ])>)

In [ ]:
import torch.nn as nn
from transformers import MobileBertModel

class MobileBertRegressor(nn.Module):
    def __init__(self, drop_rate=0.5, freeze_mobilebert=False):
        super(MobileBertRegressor, self).__init__()
        D_in, D_out = 512, 1  # Update input and output dimensions for MobileBertModel
        self.mobilebert = MobileBertModel.from_pretrained('google/mobilebert-uncased')
        self.regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(D_in, D_out))

        # Freeze MobileBertModel if specified
        if freeze_mobilebert:
            for param in self.mobilebert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_masks):
        outputs = self.mobilebert(input_ids=input_ids, attention_mask=attention_masks)
        class_label_output = outputs[1]  # Use the second element of the outputs tuple as the pooled output
        outputs = self.regressor(class_label_output)
        return outputs

# Example usage
model = MobileBertRegressor(drop_rate=0.5)

Some weights of the model checkpoint at google/mobilebert-uncased were not used when initializing MobileBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.dense.weight']
- This IS expected if you are initializing MobileBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import tensorflow as tf
from transformers import TFAutoModel

class MobileBertRegressor(tf.keras.Model):
    def __init__(self, drop_rate=0.5, freeze_mobilebert=False):
        super(MobileBertRegressor, self).__init__()
        D_in, D_out = 512, 1  # Update input and output dimensions for MobileBertModel
        self.mobilebert = TFAutoModel.from_pretrained('google/mobilebert-uncased')
        self.regressor = tf.keras.Sequential([
            tf.keras.layers.Dense(1,input_shape=(D_in))

            tf.keras.layers.Dropout(drop_rate),
            tf.keras.layers.Dense(D_out)
        ])

        # Freeze MobileBertModel if specified
        if freeze_mobilebert:
            self.mobilebert.trainable = False

    def call(self, inputs):
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']

        # Compute MobileBERT hidden states
        outputs = self.mobilebert(input_ids=input_ids, attention_mask=attention_mask)
        class_label_output = outputs[1]  # Use the second element of the outputs tuple as the pooled output

        # Pass through the regression layer
        outputs = self.regressor(class_label_output)
        return outputs

# Example usage
model = MobileBertRegressor(drop_rate=0.5)


In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer

class TFMobileBertRegressor(tf.keras.Model):
    def __init__(self, drop_rate=0.5, freeze_mobilebert=True):
        super(TFMobileBertRegressor, self).__init__()
        D_in,D_out=512,1
        self.mobilebert = TFAutoModel.from_pretrained('google/mobilebert-uncased')
        self.regressor = tf.keras.Sequential([
            tf.keras.layers.Dropout(drop_rate),
            tf.keras.layers.Dense(256, activation='relu',input_shape=(D_in,)),
            tf.keras.layers.Dropout(drop_rate),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(drop_rate),
            tf.keras.layers.Dense(D_out)
        ])

        # Freeze MobileBERT layers if specified
        if freeze_mobilebert:
            self.mobilebert.trainable = False

    def call(self, inputs):
        input_ids, attention_masks = inputs
        outputs = self.mobilebert(input_ids=input_ids, attention_mask=attention_masks)
        class_label_output = outputs[1]  # Use the second element of the outputs tuple as the pooled output
        outputs = self.regressor(class_label_output)
        return outputs
tf_model = TFMobileBertRegressor(drop_rate=0.5)

Some layers from the model checkpoint at google/mobilebert-uncased were not used when initializing TFMobileBertModel: ['predictions___cls', 'seq_relationship___cls']
- This IS expected if you are initializing TFMobileBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMobileBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFMobileBertModel were initialized from the model checkpoint at google/mobilebert-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMobileBertModel for predictions without further training.


In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")
model.to(device)

No GPU available, using the CPU instead.


MobileBertRegressor(
  (mobilebert): MobileBertModel(
    (embeddings): MobileBertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (embedding_transformation): Linear(in_features=384, out_features=512, bias=True)
      (LayerNorm): NoNorm()
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): MobileBertEncoder(
      (layer): ModuleList(
        (0-23): 24 x MobileBertLayer(
          (attention): MobileBertAttention(
            (self): MobileBertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=512, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): MobileBertSelfOutput(
              (dense): Linear(in_features=128, out_

In [ ]:
from transformers import AdamW
optimizer = AdamW(model.parameters(),
                  lr=0.001,
                  eps=1e-8)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_linear_schedule_with_warmup
epochs = 2
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                 num_warmup_steps=0, num_training_steps=total_steps)
print(total_steps)
loss_function = nn.L1Loss()

7876


In [ ]:
len(train_dataloader)

3938

In [ ]:
from torch.nn.utils.clip_grad import clip_grad_norm
def train(model, optimizer, scheduler, loss_function, epochs,
          train_dataloader, device, clip_value=2):
    for epoch in range(epochs):
        print(epoch)
        print("-----")
        best_loss = 1e10
        model.train()
        for step, batch in enumerate(train_dataloader):
            print(step)
            batch_inputs, batch_masks, batch_labels = \
             tuple(b.to(device) for b in batch)
            model.zero_grad()
            outputs = model(batch_inputs, batch_masks)
            loss = loss_function(outputs.squeeze(),
                             batch_labels.squeeze())
            loss.backward()
            clip_grad_norm(model.parameters(), clip_value)
            optimizer.step()
            scheduler.step()

    return model
model = train(tf_model, optimizer, scheduler, loss_function, epochs,
              train_dataloader, device, clip_value=2)

0
-----
0


<ipython-input-28-1b2d0dff6f96>:18: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  clip_grad_norm(model.parameters(), clip_value)


1
2


In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from sklearn.metrics import mean_absolute_error

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, epsilon=1e-8)

def create_learning_rate_scheduler(total_steps, warmup_steps):
    """
    Create a linear learning rate scheduler with warmup.
    """
    def scheduler(epoch):
        if epoch < warmup_steps:
            return (epoch + 1) / warmup_steps
        else:
            return 1.0

    return tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

epochs = 2
batch_size = 16  # Update with your desired batch size
total_steps = (len(train_dataset) // batch_size) * epochs
warmup_steps = 0  # Set warmup to 10% of total steps
loss_function = tf.keras.losses.MeanAbsoluteError()

def train(model, optimizer, loss_function, epochs, train_dataset, clip_value=2):
    for epoch in range(epochs):
        print("Epoch:", epoch + 1)
        best_loss = 1e10
        for step, batch in enumerate(train_dataset):
            print("Step:", step + 1)
            batch_inputs, batch_masks, batch_labels = batch
            with tf.GradientTape() as tape:
                outputs = model((batch_inputs, batch_masks), training=True)
                loss = loss_function(tf.squeeze(outputs), tf.squeeze(batch_labels))
            gradients = tape.gradient(loss, model.trainable_variables)
            clipped_gradients, _ = tf.clip_by_global_norm(gradients, clip_value)
            optimizer.apply_gradients(zip(clipped_gradients, model.trainable_variables))

    return model

#model = MobileBertRegressor(D_in, D_out, drop_rate=0.5, freeze_mobilebert=False)
train(tf_model, optimizer, loss_function, epochs, train_dataset, clip_value=2)


Epoch: 1
Step: 1


In [ ]:
train = pd.read_csv('/content/dataset/train.csv')
train_dropped = train.dropna()
val_dropped = train_dropped.sample(n=5000,random_state=100)

In [ ]:
encoded_val_corpus = tokenizer(text=val_dropped['TITLE'].tolist(),
                          add_special_tokens=True,
                          padding='max_length',
                          truncation='longest_first',
                          max_length=150,
                          return_attention_mask=True)
val_input_ids = np.array(encoded_val_corpus['input_ids'])
val_attention_mask = np.array(encoded_val_corpus['attention_mask'])
val_labels = val_dropped['PRODUCT_LENGTH'].to_numpy()
print(val_labels.shape)
val_dataloader = create_dataloaders(val_input_ids,
                         val_attention_mask, val_labels, batch_size)
y_pred_scaled = predict(model, val_dataloader, device)

In [ ]:
y_test = val_dropped['PRODUCT_LENGTH'].to_numpy()
y_pred = price_scaler.inverse_transform(y_pred_scaled)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)


In [ ]:
from torch.nn.utils.clip_grad import clip_grad_norm
def train(model, optimizer, scheduler, loss_function, epochs,
          train_dataloader, device, clip_value=2):
    for epoch in range(epochs):
        print(epoch)
        print("-----")
        best_loss = 1e10
        model.train()
        for step, batch in enumerate(train_dataloader):
            print(step)
            batch_inputs, batch_masks, batch_labels = [b.to(device) for b in batch]
            model.zero_grad()
            print('outputs error')
            print(batch_inputs.shape)
            print(batch_labels.shape)
            print(batch_masks.shape)
            outputs = model(batch_inputs,batch_masks)
            loss = loss_function(outputs.squeeze(), batch_labels.squeeze())
            loss.backward()
            clip_grad_norm(model.parameters(), clip_value)
            optimizer.step()
            scheduler.step()


    return model
model = train(model, optimizer, scheduler, loss_function, epochs,
              train_dataloader, device, clip_value=2)

In [ ]:
import numpy as np
from scipy.stats import yeojohnson

# Example numerical data
data = np.array([-2, -1, 0, 1, 2])

# Perform Yeo-Johnson transformation
transformed_data, _ = yeojohnson(data)

# Print the transformed data
print(transformed_data)

In [ ]:
train_dropped['PRODUCT_TYPE_ID'].head

In [ ]:
transformed_data,_ = yeojohnson(train_dropped['PRODUCT_TYPE_ID'].to_numpy())

In [ ]:
print(train_dropped['PRODUCT_TYPE_ID'].head(20))
print(transformed_data[0:20])

In [ ]:
text_cols = ['Title', 'Review Text']
cat_cols = ['Clothing ID', 'Division Name', 'Department Name', 'Class Name']
numerical_cols = ['Rating', 'Age', 'Positive Feedback Count']

column_info_dict = {
    'text_cols': text_cols,
    'num_cols': numerical_cols,
    'cat_cols': cat_cols,
    'label_col': 'Recommended IND',
    'label_list': ['Not Recommended', 'Recommended']
}


model_args = ModelArguments(
    model_name_or_path='bert-base-uncased'
)

data_args = MultimodalDataTrainingArguments(
    data_path='.',
    combine_feat_method='gating_on_cat_and_num_feats_then_sum',
    column_info=column_info_dict,
    task='classification'
)

training_args = TrainingArguments(
    output_dir="./logs/model_name",
    logging_dir="./logs/runs",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=32,
    num_train_epochs=1,
    evaluate_during_training=True,
    logging_steps=25,
    eval_steps=250
)

set_seed(training_args.seed)

### NER Preprocessing Pipeline

In [ ]:
import spacy

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

# Define a text to analyze
text = "The temperature is 25 degrees Celsius and the weight is 150 pounds."

# Process the text with SpaCy
doc = nlp(text)

# Extract numbers and units using SpaCy's named entity recognition
numbers = []
units = []
for ent in doc.ents:
    if ent.label_ == "QUANTITY":  # Check if the entity is a quantity
        number, unit = ent.text.split(" ")  # Split the quantity into number and unit
        numbers.append(float(number))  # Convert the number to float
        units.append(unit)

# Print the extracted numbers and units
print("Numbers:", numbers)
print("Units:", units)

In [ ]:
def extract_ner(data):
    quantities = []
    prices = []
    products = []
    for text in data:
        doc = nlp(text)
        for ent in doc.ents:
            if ent.label_ == "QUANTITY":  # Check if the entity is a quantity
                quantities.append(ent.text)
            elif ent.label_ == "MONEY":  # Check if the entity is a monetary value
                prices.append(ent.text)
            elif ent.label_ == "PRODUCT":  # Check if the entity is a product
                products.append(ent.text)
    return quantities, prices, products

# Example dataset
titles = ["Product Title 1", "Product Title 2", "Product Title 3"]
descriptions = ["Product description 1", "Product description 2", "Product description 3"]
bullet_points = ["Bullet point 1", "Bullet point 2", "Bullet point 3"]

# Concatenate the data into a single list
data = titles + descriptions + bullet_points

# Extract quantities, prices, and product labels
quantities, prices, products = extract_ner(data)

# Print the extracted quantities, prices, and product labels
print("Quantities:", quantities)
print("Prices:", prices)
print("Products:", products)

In [ ]:
import spacy
import numpy as np
import pandas as pd

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

# Define a function to extract quantities, prices, and product labels
def extract_ner(data):
    quantities = []
    prices = []
    products = []
    for text in data:
        doc = nlp(text)
        for ent in doc.ents:
            if ent.label_ in {"QUANTITY", "MONEY", "PRODUCT"}:  # Use set for faster membership testing
                if ent.label_ == "QUANTITY":  # Check if the entity is a quantity
                    quantities.append(ent.text)
                elif ent.label_ == "MONEY":  # Check if the entity is a monetary value
                    prices.append(ent.text)
                else:  # Entity must be a product
                    products.append(ent.text)
    return quantities, prices, products

# Example DataFrame
# Concatenate the text columns into a single Series
text_data = pd.concat([train_dropped["TITLE"], train_dropped["BULLET_POINTS"],train_dropped["DESCRIPTION"]])

# Extract quantities, prices, and product labels
quantities, prices, products = extract_ner(text_data)

# Convert the lists to NumPy arrays
quantities = np.array(quantities)
prices = np.array(prices)
products = np.array(products)

# Print the extracted quantities, prices, and product labels
print("Quantities:", quantities)
print("Prices:", prices)
print("Products:", products)

In [ ]:
import spacy
import numpy as np
import pandas as pd

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

# Define a function to extract quantities, prices, and product labels
def extract_ner(data):
    quantities = []
    prices = []
    products = []
    for doc in nlp.pipe(data, disable=["parser", "tagger"]):  # Use SpaCy's built-in text processing pipeline
        # Use NumPy vectorization to extract quantities, prices, and products
        ent_labels = np.array([ent.label_ for ent in doc.ents])
        ent_texts = np.array([ent.text for ent in doc.ents])

        # Extract quantities
        quantities_mask = ent_labels == "QUANTITY"
        quantities.extend(ent_texts[quantities_mask])

        # Extract prices
        prices_mask = ent_labels == "MONEY"
        prices.extend(ent_texts[prices_mask])

        # Extract products
        products_mask = ent_labels == "PRODUCT"
        products.extend(ent_texts[products_mask])

    return quantities, prices, products

# Example DataFrame
# Concatenate the text columns into a single Series
text_data = pd.concat([train_dropped["TITLE"], train_dropped["BULLET_POINTS"], train_dropped["DESCRIPTION"]])
# Extract quantities, prices, and product labels
quantities, prices, products = extract_ner(text_data)

# Convert the lists to NumPy arrays
quantities = np.array(quantities)
prices = np.array(prices)
products = np.array(products)

# Print the extracted quantities, prices, and product labels
print("Quantities:", quantities)
print("Prices:", prices)
print("Products:", products)


### Simple Regression with distilbert tokens

In [ ]:
# Step 1: Load the DistilBERT model and tokenizer
from transformers import DistilBertTokenizer, DistilBertModel
import torch

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Step 2: Load and preprocess the dataset
# Assume that the dataset is loaded into a pandas DataFrame called "df"
# and that the "bullet_points" column contains the bullet points and "product_length" contains the label

X = train_dropped['BULLET_POINTS'].tolist()
y = train_dropped['PRODUCT_LENGTH'].values

max_length = 512
# Preprocess the bullet points using the tokenizer
X_preprocessed = []
for text in X:
    input_ids = tokenizer.encode(text, add_special_tokens=True, padding ='max_length', max_length = max_length)
    X_preprocessed.append(input_ids)

# Step 3: Convert the preprocessed bullet points into input features
input_ids = torch.tensor(X_preprocessed)
with torch.no_grad():
    outputs = model(input_ids)
    embeddings = outputs[0]

# Step 4: Concatenate the text embeddings for each bullet point
X_concat = []
for emb in embeddings:
    X_concat.append(emb.mean(dim=0).tolist())

# Step 5: Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_concat, y, test_size=0.2, random_state=42)

# Step 6: Build the neural network
import torch.nn as nn

class RegressionModel(nn.Module):
    def _init_(self, input_size, hidden_size):
        super(RegressionModel, self)._init_()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Step 7: Train the regression model
import torch.optim as optim

model = RegressionModel(input_size=len(X_concat[0]), hidden_size=128)
criterion = nn.MAELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
batch_size = 32

for epoch in range(num_epochs):
    running_loss = 0.0
    for i in range(0, len(X_train), batch_size):
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Forward + backward + optimize
        outputs = model(torch.tensor(X_train[i:i+batch_size]).float())
        loss = criterion(outputs.squeeze(), torch.tensor(y_train[i:i+batch_size]).float())
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item() * len(X_train[i:i+batch_size])
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(X_train):.4f}')

# Step 8: Evaluate the performance of the regression model
from sklearn.metrics import mean_squared_error, mean_absolute_error

with torch.no_grad():
    y_pred = model(torch.tensor(X_test).float()).squeeze().numpy()

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'MSE: {mse:.4f}')
print(f'MAE: {mae:.4f}')

# Step 9: Make predictions on new, unseen data
# Assume that we have a new dataset loaded into a pandas DataFrame called "new_df"
# and that the "bullet_points" column contains the bullet points we want to predict the length of

# Preprocess the bullet points using the tokenizer
X_new = trial_test['BULLET_POINTS'].tolist()
X_new_preprocessed = []
for text in X_new:
    input_ids = tokenizer.encode(text, add_special_tokens=True)
    X_new_preprocessed.append(input_ids)

# Convert the preprocessed bullet points into input features
input_ids = torch.tensor(X_new_preprocessed)
with torch.no_grad():
    outputs = model(input_ids)
    embeddings = outputs[0]

# Concatenate the text embeddings for each bullet point
X_new_concat = []
for emb in embeddings:
    X_new_concat.append(emb.mean(dim=0).tolist())

# Make predictions using the trained regression model
with torch.no_grad():
    y_pred_new = model(torch.tensor(X_new_concat).float()).squeeze().numpy()

# Print the predicted product lengths for each bullet point
for i in range(len(trial_test)):
    print(f'Bullet point: {X_new[i]}')
    print(f'Predicted length: {y_pred_new[i]:.2f}')
    print('-' * 50)

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
max_length = 128
max_seq_length = 128
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Step 2: Load and preprocess the dataset
# Assume that the dataset is loaded into a pandas DataFrame called "df"
# and that the "bullet_points" column contains the bullet points and "product_length" contains the label

X = train_dropped['BULLET_POINTS'].tolist()
y = train_dropped['PRODUCT_LENGTH'].values

# Preprocess the bullet points using the tokenizer
X_preprocessed = []
for text in X:
    input_ids = tokenizer.encode(text, add_special_tokens=True,padding = 'max_length',max_length = max_length,truncation = True) #Truncation done
    X_preprocessed.append(input_ids)
    # Pad or truncate the input sequence to a fixed length
    if len(input_ids) > max_seq_length:
        input_ids = input_ids[:max_seq_length-1] + [tokenizer.sep_token_id]  # Truncate and add SEP token
    else:
        padding_length = max_seq_length - len(input_ids)
        input_ids = input_ids + [tokenizer.pad_token_id] * padding_length

In [ ]:
len(input_ids)

In [ ]:
len(X_preprocessed[1000])

In [ ]:

max_seq_length = 512  # or another value that matches the model's maximum sequence length
batch_size = 64  # for example
X_concat = []
model.eval()
with torch.no_grad():
    for i in range(0, len(X_preprocessed), batch_size):
        input_ids = torch.tensor(X_preprocessed[i:i+batch_size]).to(device)
        # Truncate the input sequences to the maximum sequence length
        input_ids = input_ids[:,:max_seq_length]
        outputs = model(input_ids)
        embeddings = outputs[0]
        for emb in embeddings:
            X_concat.append(emb.mean(dim=0).tolist())

In [ ]:
input_ids = torch.tensor(X_preprocessed).long()

In [ ]:
input_ids.shape

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error

def mean_absolute_percentage_error(y_true, y_pred):
    """
    Calculate the mean absolute percentage error (MAPE) between y_true and y_pred.
    Args:
        y_true (array-like): Array of true labels.
        y_pred (array-like): Array of predicted labels.
    Returns:
        float: Mean absolute percentage error.
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))

# Create some example true labels and predicted labels
y_true = [1.2, 2.5, 3.8, 4.1, 5.3]
y_pred = [1.0, 2.4, 4.0, 4.2, 5.2]

# Calculate the mean absolute percentage error
mape = mean_absolute_percentage_error(y_true, y_pred)

# Calculate the score using the formula
score = max(0, 100*(1 - mape))

# Print the score
print("Score:", score)


In [ ]:
# Step 1: Load the DistilBERT model and tokenizer
 # Pad with PAD tokens
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Step 3: Convert the preprocessed bullet points into input features
#input_ids = torch.tensor(input_ids).to(device).long()
input_ids = torch.tensor(X_preprocessed).long()
model.to(device)
with torch.no_grad():
    outputs = model(input_ids)
    embeddings = outputs[0]

# Step 4: Concatenate the text embeddings for each bullet point
X_concat = []
for emb in embeddings:
    X_concat.append(emb.mean(dim=0).tolist())

# Step 5: Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_concat, y, test_size=0.2, random_state=42)

# Step 6: Build the neural network
import torch.nn as nn

class RegressionModel(nn.Module):
    def _init_(self, input_size, hidden_size):
        super(RegressionModel, self)._init_()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Step 7: Train the regression model
import torch.optim as optim

model = RegressionModel(input_size=len(X_concat[0]), hidden_size=128)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
batch_size = 32

for epoch in range(num_epochs):
    running_loss = 0.0
    for i in range(0, len(X_train), batch_size):
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(torch.tensor(X_train[i:i+batch_size]).float())
        loss = criterion(outputs.squeeze(), torch.tensor(y_train[i:i+batch_size]).float())
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item() * len(X_train[i:i+batch_size])
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(X_train):.4f}')

# Step 8: Evaluate the performance of the regression model
from sklearn.metrics import mean_squared_error, mean_absolute_error

with torch.no_grad():
    y_pred = model(torch.tensor(X_test).float()).squeeze().numpy()

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'MSE: {mse:.4f}')
print(f'MAE: {mae:.4f}')

# Step 9: Make predictions on new, unseen data
# Assume that we have a new dataset loaded into a pandas DataFrame called "new_df"
# and that the "bullet_points" column contains the bullet points we want to predict the length of

# Preprocess the bullet points using the tokenizer
X_new = trial_test['BULLET_POINTS'].tolist()
X_new_preprocessed = []
for text in X_new:
    input_ids = tokenizer.encode(text, add_special_tokens=True)
    X_new_preprocessed.append(input_ids)

# Convert the preprocessed bullet points into input features
input_ids = torch.tensor(X_new_preprocessed)
with torch.no_grad():
    outputs = model(input_ids)
    embeddings = outputs.last_hidden_state

# Concatenate the text embeddings for each bullet point
X_new_concat = []
for emb in embeddings:
    X_new_concat.append(emb.mean(dim=0).tolist())

# Make predictions using the trained regression model
with torch.no_grad():
    y_pred_new = model(torch.tensor(X_new_concat).float()).squeeze().numpy()

# Print the predicted product lengths for each bullet point
for i in range(len(trial_test)):
    print(f'Bullet point: {X_new[i]}')
    print(f'Predicted length: {y_pred_new[i]:.2f}')
    print('-' * 50)

In [ ]:
from transformers import DistilBertTokenizer

# Create a DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Example input text
input_text = "This is an example sentence for tokenization lego batman outfits cosplay tyler durden."

# Tokenize the input text
input_ids = tokenizer.encode(input_text, add_special_tokens=True)

# Specify the maximum allowed sequence length
max_seq_length = 10

# Truncate or pad the input_ids token sequence to ensure it adheres to max_seq_length
if len(input_ids) > max_seq_length:
    input_ids = input_ids[:max_seq_length-1] + [tokenizer.sep_token_id]  # Truncate and add SEP token
else:
    padding_length = max_seq_length - len(input_ids)
    input_ids = input_ids + [tokenizer.pad_token_id] * padding_length  # Pad with PAD tokens

# Print the resulting input_ids token sequence
print(input_ids)

###Misc

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
corr = train['PRODUCT_TYPE_ID'].corr(train['PRODUCT_LENGTH'])
sns.scatterplot(x='PRODUCT_TYPE_ID', y='PRODUCT_LENGTH', data=train)
sns.regplot(x='PRODUCT_TYPE_ID', y='PRODUCT_LENGTH', data=train, scatter=False)
plt.title(f'Correlation: {corr:.2f}')
plt.show()